In [ ]:
import os
import pandas as pd
import numpy as np
import random
import holidays
import plotly.express as px
import matplotlib.pyplot as plt 
import plotly.graph_objects as go

from tqdm import tqdm
from scipy.signal import find_peaks, detrend

import plotly.express as px 
from azureml.core import Workspace, Dataset
from IPython.display import display



from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor 

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.metrics import mean_squared_error 

from sklearn.metrics import mean_absolute_percentage_error

from sklearn.linear_model import LinearRegression

In [ ]:
def describe_time_series(df , col = None , hist_margin = 'rug' , hist_on = False ) :
    ### takes a dataframe with a datetime index
    ### returns
    ##  summary statistics 
    ## plots 

    col_ts =  col if col != None else df.columns[0] 
    ts = df.loc[:, [col_ts] ]

    ## CHECK TIMESTAMPS
    continous_index = pd.date_range ( ts.index.min() , ts.index.max() ,  freq = ts.index.freq)
    continous_dates_df = pd.DataFrame(index = continous_index ,
                                      data = ({'cont_dates': continous_index }) )
    
    measures_per_timestamp = continous_dates_df.merge(ts , left_index= True , right_index= True , how = 'left').groupby('cont_dates').count()
    measures_per_timestamp['problem'] = measures_per_timestamp[measures_per_timestamp != 1 ] 

    print(' DUPLICATE OR MISSING TIMESTAMPS' , measures_per_timestamp[measures_per_timestamp.problem == 1 ])   

    ## CHECK NULL VALUES 
    print ('MISSING VALUES in ', ts.isnull().sum()[0], ',', (ts.isnull().sum()/ts.shape[0])[0] , ' %  TIMESTEPS' ) 
    print( 'NUMBER OF TIMESTEPS' , ts.shape[0])

    ## PLOT TIMESERIES 
    line_plot  = px.line( data_frame= ts  , 
          x = ts.index , 
          y = str(col_ts) , 
          title = 'LINE PLOT ' + str(col_ts) )

    ## DISTRIBUTION PLOT 

    histo = px.histogram( ts, x= str(col_ts),  marginal=hist_margin ) # can be `box`, `violin`
    ## CHECK 

    yearly_summary = ts.resample('Y').agg( ['sum' ,'mean','min' , 'max' , 'std'])
    
    line_plot.show()
    if hist_on == True: 
          histo.show()

    print(yearly_summary)



    return 
    


In [ ]:
def extract_time_features( df_in , holidays_on = True , feature_format = 'linear' ): 
    ## takes a dataframe with the index as a timeseries 
    ## gets back time related features in the dataframe 
    ## parameters : feature format : linear or sinusoidal 


    df = df_in.copy(deep=True) 

    df['is_weekend'] = (df.index.weekday > 4).astype(int)

    if holidays_on == True:
        country_holidays = holidays.Belgium()
        df['dates'] = df.index 
        df['is_holiday'] = df.dates.apply( lambda x: x in country_holidays).astype(int)
        df.drop( columns = 'dates' , inplace = True )

        

    if feature_format == 'linear': 
        df['hour'] =    df.index.hour
        df['month'] =   df.index.month
        df['weekday'] = df.index.weekday

    if feature_format == 'sinusoidal':
        df['hour_cos'] = np.cos( ( df.index.hour +1 ) / 24 * 2 * np.pi )
        df['hour_sin'] = np.sin( ( df.index.hour +1 ) / 24 * 2 * np.pi )

        df['month_cos'] = np.cos( df.index.month / 12 * 2 * np.pi )
        df['month_sin'] = np.sin( df.index.month / 12 * 2 * np.pi )

        
        df['weekday_cos'] = np.cos( (df.index.weekday + 1 ) / 7 * 2 * np.pi )
        df['weekday_sin'] = np.sin( (df.index.weekday + 1 ) / 7 * 2 * np.pi )

    return df 



In [ ]:
def create_lags(df_in , col = None , n_lags = [ 96 , 96*2 ] , na_strategy = 'bfill' , verbose = False  ):

    ## Takes a dataframe and a column and a list of lags to be performed 
    ## strategy to handle na_shall_also_be_specified 

    df = df_in.copy(deep = True )
 
    col_ts =  col if col != None else df.columns[0] 

    for l in n_lags:

        df['lag_'+str(l)+'_'+col_ts] = df[col_ts].shift(l)
        
    df.fillna( method = na_strategy  , inplace = True )
    if verbose == True : 
        print(df.columns)

    return df 
    

In [ ]:
def plot_all_columns( df_input , graph_title  ) :
    fig = go.Figure()

    for c in df_input.columns:
        fig.add_trace( go.Scatter( x = df_input.index , y = df_input[c] , name = str(c) ))

    fig.update_layout(title = graph_title)
    return fig.show()

In [ ]:
##df_index = pd.date_range('2019-01-01' ,'2022-01-01' , freq = '15min' )  
##df_values = 10*( np.sin( df_index.hour ))

##df = pd.DataFrame( index = df_index , data = {'measure' : 1000*( np.sin( df_index.hour )),
##                                            'temperature' : 4*( np.cos( df_index.hour )) })           


In [ ]:
PAR= { 'TARGET_VALUE' : 'consumption_mw' , 
       'TRAIN_START' : '2019-01-01' , 
       'TRAIN_END'   :  '2022-01-01' , 
       'TEST_END' :     '2022-12-31'}

# PROBLEM FORMULATION

### Construire un modele pour tous les valeurs de consommations 15min de demain 


### Le dataset

Elia c'est une gestionnaire de réseau belge qui mets à dispositon des données en libre acces 

https://opendata.elia.be/pages/home/ 

Dataset de consommation (MW)
https://opendata.elia.be/explore/dataset/ods001/information/?sort=datetime  


In [ ]:
load = pd.read_csv( 'https://opendata.elia.be/explore/dataset/ods001/download/?format=csv&q=datetime%3E%3D%222019-12-31T23:00:00Z%22&timezone=Europe/Brussels&lang=fr&use_labels_for_header=true&csv_separator=%3B', sep = ';')
load.head()

df = pd.DataFrame( index = pd.date_range ('2020-01-01 00:00:00' , 	str(pd.Timestamp.today().date()) +' 23:45:00' , freq = '15min' ) , 
                    data = {'consumption_mw': load.sort_values( 'DateTime' )['Total Load'].values  }  , 
                  ).dropna()

In [ ]:
df.head()

In [ ]:
describe_time_series(df)

## Facteurs de dependence

In [ ]:
prep_df  =( df.pipe(  extract_time_features , feature_format = 'sinusoidal' )
              .pipe(  create_lags , col = PAR['TARGET_VALUE'], n_lags = [96*2] ))
 

In [ ]:
prep_df.head()

In [ ]:
plot_all_columns(prep_df[prep_df.index > '2020-04-01 00:00:00'].head(96 * 7)  , graph_title= 'Features pour une semaine ')

In [ ]:
px.bar( prep_df.corr()['consumption_mw'].sort_values() , title = 'Correlations avec consumption MW' )

In [ ]:

#df['temperature'] = 12 + 10 * np.cos( df.index.month / 12 * 2 * np.pi ) + 3 *  np.sin( df.index.hour / 24 * 2 * np.pi ) + 3*random.uniform(-1, 1)
#describe_time_series( df , col = 'temperature')

# Separer dataset train et test 

In [ ]:
train  = prep_df[ (prep_df.index >= PAR['TRAIN_START']) &  (prep_df.index <= PAR['TRAIN_END'])]
test   = prep_df[ (df.index > PAR['TRAIN_END'] )  & ( prep_df.index < PAR['TEST_END'])]

y_train  = train[PAR['TARGET_VALUE']]
X_train  = train.drop(columns = PAR['TARGET_VALUE'] ) 

y_test  = test[PAR['TARGET_VALUE']]
X_test  = test.drop(columns = PAR['TARGET_VALUE'] ) 


In [ ]:
fig = go.Figure() 
fig.add_trace( go.Scatter( x = y_train.index , y = y_train.values , name = 'Train'))
fig.add_trace( go.Scatter( x = y_test.index ,  y = y_test.values ,  name = 'Test'))
fig.update_layout( title = 'TRAIN - TEST')
fig.show()

###

# Avant des modeles complexes: une baseline solide ! 
On note une periodicité horaire et une difference jour et jour 
un modele simple mais efficace pourrait etre que la consommation de demain à l'heure X est la consommation il y a 7 jour  à la meme heure


In [ ]:
y_naive= prep_df[PAR['TARGET_VALUE']].shift( 96 * 7 )
y_naive_test = y_naive[y_naive.index.isin(y_test.index)]

In [ ]:
fig = go.Figure() 

fig.add_trace( go.Scatter( x = y_test.index ,  y = y_test ,  name = 'Test'))
fig.add_trace( go.Scatter( x = y_naive_test.index ,  y = y_naive_test ,  name = 'NAIVE 7 jours'))

fig.update_layout(title = 'NAIVE MODEL VS TEST SET')
fig.show()

In [ ]:
print( np.round( 100* np.mean( np.abs( y_naive_test - y_test ) / y_test ) ,3 ) , 'MAPE NAIVE 7 jours' )  

In [ ]:
fig_hist = go.Figure() 
fig_hist = px.histogram(x =  y_naive_test - y_test  , title = 'Distribution des erreurs' )
fig_hist.add_trace( go.Scatter( x = [0,0] , y = [0, 1000]))

## Augmenter la complexité Graduellement

1. Modele de regression lineaire
2. Modele Random Forest

D'autres choix sont possibles:
- Travailler sur les parametres des modeles 
- Travailler sur la forme du probleme en time serie 
- Travailler avec des modeles plus complexes 

In [ ]:
linear_regression_pipeline = Pipeline(
[
('scaler' ,        StandardScaler() ), 
('LR' ,            LinearRegression())] ) 

linear_regression_pipeline.fit(X_train , y_train )

In [ ]:
random_forest_pipeline = Pipeline(
[
('scaler' ,        StandardScaler() ), 
('RF' ,            RandomForestRegressor(n_estimators=50 ))] ) 

random_forest_pipeline.fit(X_train , y_train )

In [ ]:
random_forest_pipeline.predict(X_test)

## Work on the model parameters

In [ ]:
cv_params = { 'RF__max_depth' : [2, 5, 8 , 10 , 12] , 
             'RF__n_estimators' : [30 , 60 , 90 , 120 ]  }

ts_cv  = TimeSeriesSplit(n_splits = 2 , gap = 96 ) 

In [ ]:
grid_search_model = GridSearchCV( 
                            estimator= random_forest_pipeline , 
                            param_grid= cv_params, 
                            cv = ts_cv  , 
                            scoring = make_scorer(mean_squared_error, greater_is_better= False) ,
                            return_train_score = True , 
                            verbose = 4, ) 

In [ ]:
grid_search_model.fit(X_train, y_train)

In [ ]:
grid_search_model.best_params_

## Compute Predictions

In [ ]:
results = pd.DataFrame(y_test)
results['pred_naive_'+PAR['TARGET_VALUE']] = y_naive_test 
results['pred_lr_'+PAR['TARGET_VALUE']] = linear_regression_pipeline.predict(X_test)
results['pred_rf_'+PAR['TARGET_VALUE'] ] = random_forest_pipeline.predict(X_test)
results['pred_rf_cv_'+PAR['TARGET_VALUE'] ] = grid_search_model.predict(X_test)

In [ ]:
results.head()

In [ ]:
plot_all_columns(results , graph_title = 'Predictions')

## Metriques

In [ ]:
models = []
mape_error = []
rmse_error  =[]
mse_error = []

for c in results.columns[1:]:
    models.append( c )
    mape_error.append( np.round( 100*mean_absolute_percentage_error( results[PAR['TARGET_VALUE']].values , results[c].values ) ,  3 )) 
    rmse_error.append( np.round( mean_squared_error( results[PAR['TARGET_VALUE']].values , results[c].values , squared = True ) , 3 ))
    mse_error.append(  np.round( mean_squared_error( results[PAR['TARGET_VALUE']].values , results[c].values , squared = False) , 3 ))

metrics_df = pd.DataFrame( data = {'models' : models, 
                             'mape': mape_error , 
                             'rmse':rmse_error,
                             'mse': mse_error })

# Metriques 

In [ ]:
metrics_df.style.highlight_min(subset = ['mape' , 'rmse' ,'mse'])

In [ ]:
fig_hist = go.Figure() 
fig_hist = px.histogram(x =  results[PAR['TARGET_VALUE']].values - results['pred_rf_cv_consumption_mw']  , title = 'Distribution des erreurs' )
fig_hist.add_trace( go.Scatter( x = [0,0] , y = [0, 1000]))